In [352]:
import pandas as pd
import numpy as np

from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC

from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV

In [353]:
df_vehicle = pd.read_csv('Data/Train Data/Train_Vehicle.csv')
df_claim = pd.read_csv('Data/Train Data/Train_Claim.csv')
df_demo = pd.read_csv('Data/Train Data/Train_Demographics.csv')
df_policy = pd.read_csv('Data/Train Data/Train_Policy.csv')
df_target = pd.read_csv('Data/Train Data/Traindata_with_Target.csv')

In [354]:
df_target

,CustomerID,ReportedFraud
0,Cust20065,N
1,Cust37589,N
2,Cust24312,N
3,Cust5493,Y
4,Cust7704,Y
...,...,...
28831,Cust9773,N
28832,Cust38439,N
28833,Cust38444,N
28834,Cust26466,N


In [355]:
df_policy

,InsurancePolicyNumber,CustomerLoyaltyPeriod,DateOfPolicyCoverage,InsurancePolicyState,Policy_CombinedSingleLimit,Policy_Deductible,PolicyAnnualPremium,UmbrellaLimit,InsuredRelationship,CustomerID
0,110122,328,2014-10-17,State3,250/500,1000,1406.91,0,husband,Cust1001
1,110125,256,1990-05-25,State1,250/500,2000,1415.74,6000000,unmarried,Cust1004
2,110126,228,2014-06-06,State1,500/1000,1000,1583.91,6000000,unmarried,Cust1005
3,110127,256,2006-10-12,State3,250/500,1000,1351.10,0,unmarried,Cust1006
4,110128,137,2000-06-04,State2,250/500,1000,1333.35,0,husband,Cust1007
...,...,...,...,...,...,...,...,...,...,...
28831,148614,425,1999-10-12,State3,250/500,1000,839.81,0,wife,Cust39493
28832,148615,367,1996-11-09,State3,250/500,1000,811.03,0,husband,Cust39494
28833,148616,208,1995-12-04,State3,250/500,1000,891.37,0,husband,Cust39495
28834,148618,401,1994-09-12,State3,250/500,856,911.37,0,own-child,Cust39497


In [356]:
df_demo

,CustomerID,InsuredAge,InsuredZipCode,InsuredGender,InsuredEducationLevel,InsuredOccupation,InsuredHobbies,CapitalGains,CapitalLoss,Country
0,Cust10000,35,454776,MALE,JD,armed-forces,movies,56700,-48500,India
1,Cust10001,36,454776,MALE,JD,tech-support,cross-fit,70600,-48500,India
2,Cust10002,33,603260,MALE,JD,armed-forces,polo,66400,-63700,India
3,Cust10003,36,474848,MALE,JD,armed-forces,polo,47900,-73400,India
4,Cust10004,29,457942,FEMALE,High School,exec-managerial,dancing,0,-41500,India
...,...,...,...,...,...,...,...,...,...,...
28831,Cust9993,46,600561,MALE,Masters,protective-serv,sleeping,0,0,India
28832,Cust9994,44,439304,MALE,Masters,adm-clerical,dancing,0,-28800,India
28833,Cust9996,53,460722,MALE,PhD,transport-moving,base-jumping,63100,-43800,India
28834,Cust9997,53,472634,MALE,Associate,transport-moving,skydiving,67400,-43800,India


In [357]:
df_claim

,CustomerID,DateOfIncident,TypeOfIncident,TypeOfCollission,SeverityOfIncident,AuthoritiesContacted,IncidentState,IncidentCity,IncidentAddress,IncidentTime,NumberOfVehicles,PropertyDamage,BodilyInjuries,Witnesses,PoliceReport,AmountOfTotalClaim,AmountOfInjuryClaim,AmountOfPropertyClaim,AmountOfVehicleDamage
0,Cust10000,2015-02-03,Multi-vehicle Collision,Side Collision,Total Loss,Police,State7,City1,Location 1311,17,3,?,1,0,?,65501,13417,6071,46013
1,Cust10001,2015-02-02,Multi-vehicle Collision,Side Collision,Total Loss,Police,State7,City5,Location 1311,10,3,YES,2,1,YES,61382,15560,5919,39903
2,Cust10002,2015-01-15,Single Vehicle Collision,Side Collision,Minor Damage,Other,State8,City6,Location 2081,22,1,YES,2,3,NO,66755,11630,11630,43495
3,Cust10003,2015-01-19,Single Vehicle Collision,Side Collision,Minor Damage,Other,State9,City6,Location 2081,22,1,YES,2,3,NO,66243,12003,12003,42237
4,Cust10004,2015-01-09,Single Vehicle Collision,Rear Collision,Minor Damage,Fire,State8,City6,Location 1695,10,1,NO,2,1,YES,53544,8829,7234,37481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28831,Cust9993,2015-01-24,Vehicle Theft,?,Trivial Damage,Police,State9,City4,Location 1890,3,1,NO,0,3,?,6590,870,1078,4642
28832,Cust9994,2015-02-09,Single Vehicle Collision,Front Collision,Minor Damage,Fire,State8,City3,Location 2097,17,1,YES,0,2,YES,74547,14699,7875,51973
28833,Cust9996,2015-01-28,Single Vehicle Collision,Rear Collision,Minor Damage,Fire,State9,City7,Location 1452,1,1,YES,2,0,YES,55305,7043,7043,41219
28834,Cust9997,2015-01-28,Single Vehicle Collision,Rear Collision,Minor Damage,Ambulance,State9,City3,Location 1876,1,1,?,2,0,YES,55830,7059,7059,41712


In [358]:
df_vehicle

,CustomerID,VehicleAttribute,VehicleAttributeDetails
0,Cust20179,VehicleID,Vehicle8898
1,Cust21384,VehicleModel,Malibu
2,Cust33335,VehicleMake,Toyota
3,Cust27118,VehicleModel,Neon
4,Cust13038,VehicleID,Vehicle30212
...,...,...,...
115339,Cust22134,VehicleMake,Audi
115340,Cust25338,VehicleID,Vehicle28831
115341,Cust7637,VehicleID,Vehicle12870
115342,Cust33581,VehicleID,Vehicle19680


In [359]:
df_vehicle = df_vehicle.pivot(index="CustomerID", columns="VehicleAttribute", values="VehicleAttributeDetails").reset_index()

In [360]:
df_vehicle

VehicleAttribute,CustomerID,VehicleID,VehicleMake,VehicleModel,VehicleYOM
0,Cust10000,Vehicle26917,Audi,A5,2008
1,Cust10001,Vehicle15893,Audi,A5,2006
2,Cust10002,Vehicle5152,Volkswagen,Jetta,1999
3,Cust10003,Vehicle37363,Volkswagen,Jetta,2003
4,Cust10004,Vehicle28633,Toyota,CRV,2010
...,...,...,...,...,...
28831,Cust9993,Vehicle13568,Suburu,Impreza,2007
28832,Cust9994,Vehicle19810,Jeep,Wrangler,2003
28833,Cust9996,Vehicle3339,Suburu,Legacy,2004
28834,Cust9997,Vehicle10240,Suburu,Forrestor,2004


In [361]:
df_vehicle['CustomerID'].nunique()

28836

In [362]:
df_vehicle.drop_duplicates(subset='CustomerID', inplace=True)

# Analysis

In [363]:
df['ReportedFraud'].value_counts(normalize=True)

N    0.730025
Y    0.269975
Name: ReportedFraud, dtype: float64

In [364]:
df_target['ReportedFraud'].replace({'N':0, 'Y':1}, inplace=True)

In [365]:
df_vehicle['VehicleID'].nunique()

28836

In [366]:
df_vehicle['VehicleMake'].unique()

array(['Audi', 'Volkswagen', 'Toyota', 'Mercedes', 'Suburu', 'Saab',
       'Nissan', 'Ford', 'Accura', 'Dodge', 'Honda', 'Chevrolet', 'Jeep',
       'BMW', '???'], dtype=object)

In [367]:
df_vehicle.replace({'???':np.NaN}, inplace=True)

In [368]:
make = {'Audi':1, 'Volkswagen':2, 'Toyota':3, 'Mercedes':4, 'Suburu':5, 'Saab':6,
       'Nissan':7, 'Ford':8, 'Accura':9, 'Dodge':10, 'Honda':11, 'Chevrolet':12, 'Jeep':13,
       'BMW':14}

In [369]:
df_vehicle['VehicleMake'].replace(make, inplace=True)

In [370]:
df_vehicle['VehicleModel'].unique()

array(['A5', 'Jetta', 'CRV', 'C300', 'Passat', '92x', 'Ultima', 'Fusion',
       'Impreza', '93', 'Highlander', 'X5', 'Accord', 'Corolla',
       'Forrestor', 'F150', 'Pathfinder', 'Neon', 'Tahoe', 'Wrangler',
       'A3', 'RSX', 'Malibu', 'E400', 'Legacy', '95', 'Grand Cherokee',
       'Escape', 'Civic', 'Silverado', 'RAM', 'Camry', 'M5', '3 Series',
       'ML350', 'Maxima', 'MDX', 'X6', 'TL'], dtype=object)

In [371]:
df_vehicle.drop(columns='VehicleModel', inplace=True)

In [372]:
df_vehicle['Years_from_man'] = datetime.now().year - pd.to_datetime(df_vehicle['VehicleYOM']).dt.year

In [373]:
df_vehicle.drop(columns='VehicleYOM', inplace=True)

In [374]:
df_vehicle.drop(columns='VehicleID', inplace=True)

In [375]:
df_claim['MonthOfIncident'] = pd.to_datetime(df_claim['DateOfIncident']).dt.month

In [376]:
df_claim['TypeOfIncident'].unique()

array(['Multi-vehicle Collision', 'Single Vehicle Collision',
       'Parked Car', 'Vehicle Theft'], dtype=object)

In [377]:
df_claim['TypeOfCollission'].unique()

array(['Side Collision', 'Rear Collision', 'Front Collision', '?'],
      dtype=object)

In [378]:
df_claim.replace({'?':np.NaN}, inplace=True)

In [379]:
df_claim['TypeOfCollission'].unique()

array(['Side Collision', 'Rear Collision', 'Front Collision', nan],
      dtype=object)

In [380]:
df_claim['SeverityOfIncident'].unique()

array(['Total Loss', 'Minor Damage', 'Major Damage', 'Trivial Damage'],
      dtype=object)

In [381]:
severity = {'Trivial Damage':1, 'Minor Damage':2, 'Major Damage':3,'Total Loss':4 }

In [382]:
df_claim['SeverityOfIncident'].replace(severity, inplace=True)

In [383]:
df_claim['AuthoritiesContacted'].unique()

array(['Police', 'Other', 'Fire', 'Ambulance', 'None'], dtype=object)

In [384]:
df_claim['IncidentState'] = df_claim['IncidentState'].astype(str).apply(lambda x: x[-1] if len(x)==6 else None)

In [385]:
df_claim['IncidentCity'] = df_claim['IncidentCity'].astype(str).apply(lambda x: x[-1] if len(x)==5 else None)

In [386]:
df_claim.replace({'YES':1, 'NO':0}, inplace=True)

In [387]:
df_claim['Witnesses'].replace({'MISSINGVALUE':np.NaN}, inplace=True)

In [388]:
df_claim['Witnesses'] = df_claim['Witnesses'].astype('Int64')

In [389]:
ohe_cols = ['TypeOfIncident','TypeOfCollission','AuthoritiesContacted']

In [390]:
df_claim = pd.get_dummies(df_claim, prefix= ohe_cols, columns= ohe_cols, drop_first=True)

In [391]:
df_claim.drop(columns=['DateOfIncident','IncidentAddress'], inplace=True)

In [392]:
df_claim[['IncidentState','IncidentCity']] = df_claim[['IncidentState','IncidentCity']].astype('int64')

In [393]:
df_claim['AmountOfTotalClaim'].replace({'MISSEDDATA':np.NaN}, inplace=True)

In [394]:
df_claim['AmountOfTotalClaim'] = df_claim['AmountOfTotalClaim'].astype('float64')

In [395]:
df_claim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28836 entries, 0 to 28835
Data columns (total 24 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   CustomerID                               28836 non-null  object 
 1   SeverityOfIncident                       28836 non-null  int64  
 2   IncidentState                            28836 non-null  int64  
 3   IncidentCity                             28836 non-null  int64  
 4   IncidentTime                             28836 non-null  int64  
 5   NumberOfVehicles                         28836 non-null  int64  
 6   PropertyDamage                           18377 non-null  float64
 7   BodilyInjuries                           28836 non-null  int64  
 8   Witnesses                                28790 non-null  Int64  
 9   PoliceReport                             19031 non-null  float64
 10  AmountOfTotalClaim                       28786

In [396]:
df_demo['InsuredGender'].replace({'MALE':1, "FEMALE":0}, inplace=True)

In [397]:
df_demo['InsuredEducationLevel'].unique()

array(['JD', 'High School', 'Masters', 'MD', 'Associate', 'College',
       'PhD'], dtype=object)

In [398]:
education = {'High School':1, 'College':2, 'Associate':3, 'Masters':4, 'MD':5, 'PhD':6, 'JD':7}

In [399]:
df_demo['InsuredEducationLevel'].replace(education, inplace=True)

In [400]:
ohe_cols_demo = ['InsuredOccupation','InsuredHobbies']

In [401]:
df_demo = pd.get_dummies(df_demo, prefix= ohe_cols_demo, columns= ohe_cols_demo, drop_first=True)

In [402]:
df_demo.drop(columns=['Country''InsuredZipCode'], inplace=True)

In [403]:
df_policy['DateOfPolicyCoverage'] = pd.to_datetime(df_policy['DateOfPolicyCoverage']).dt.year

In [404]:
df_policy['YearFromPolicyCoverage'] = datetime.now().year - df_policy['DateOfPolicyCoverage']

In [405]:
df_policy['InsurancePolicyState'] = df_policy['InsurancePolicyState'].astype(str).apply(lambda x: x[-1] if len(x)==6 else None)

In [406]:
df_policy['InsurancePolicyState'] = df_policy['InsurancePolicyState'].astype('int64')

In [407]:
df_policy['InsuredRelationship'].unique()

array(['husband', 'unmarried', 'own-child', 'wife', 'other-relative',
       'not-in-family'], dtype=object)

In [408]:
df_policy[['Limit_1','Limit_2']] = df_policy['Policy_CombinedSingleLimit'].apply(lambda x: pd.Series(str(x).split("/")))

In [409]:
df_policy['Limit_1'] = df_policy['Limit_1'].astype('int64')
df_policy['Limit_2'] = df_policy['Limit_2'].astype('int64')

In [410]:
df_policy['Policy_CombinedSingleLimit'] = df_policy['Limit_1']/df_policy['Limit_2']

In [411]:
df_policy.drop(columns=['DateOfPolicyCoverage','Limit_1','Limit_2'], inplace=True)

In [412]:
df_policy.drop(columns='InsurancePolicyNumber', inplace=True)

In [413]:
df_policy

,CustomerLoyaltyPeriod,InsurancePolicyState,Policy_CombinedSingleLimit,Policy_Deductible,PolicyAnnualPremium,UmbrellaLimit,InsuredRelationship,CustomerID,YearFromPolicyCoverage
0,328,3,0.5,1000,1406.91,0,husband,Cust1001,9
1,256,1,0.5,2000,1415.74,6000000,unmarried,Cust1004,33
2,228,1,0.5,1000,1583.91,6000000,unmarried,Cust1005,9
3,256,3,0.5,1000,1351.10,0,unmarried,Cust1006,17
4,137,2,0.5,1000,1333.35,0,husband,Cust1007,23
...,...,...,...,...,...,...,...,...,...
28831,425,3,0.5,1000,839.81,0,wife,Cust39493,24
28832,367,3,0.5,1000,811.03,0,husband,Cust39494,27
28833,208,3,0.5,1000,891.37,0,husband,Cust39495,28
28834,401,3,0.5,856,911.37,0,own-child,Cust39497,29


In [414]:
ohe_cols_policy = ['InsuredRelationship']

In [415]:
df_policy = pd.get_dummies(df_policy, prefix= ohe_cols_policy, columns= ohe_cols_policy, drop_first=True)

# Merge

In [56]:
#df = df_vehicle.merge(df_target, how='inner', on='CustomerID')

In [58]:
#df = df.merge(df_claim, how='inner', on='CustomerID')

In [60]:
#df = df.merge(df_demo, how='inner', on='CustomerID')

In [61]:
#df.merge(df_policy, how='inner', on='CustomerID')

,CustomerID,VehicleID,VehicleMake,VehicleModel,VehicleYOM,ReportedFraud,DateOfIncident,TypeOfIncident,TypeOfCollission,SeverityOfIncident,...,Country,InsurancePolicyNumber,CustomerLoyaltyPeriod,DateOfPolicyCoverage,InsurancePolicyState,Policy_CombinedSingleLimit,Policy_Deductible,PolicyAnnualPremium,UmbrellaLimit,InsuredRelationship
0,Cust10000,Vehicle26917,Audi,A5,2008,N,2015-02-03,Multi-vehicle Collision,Side Collision,Total Loss,...,India,119121,49,1998-10-25,State1,100/300,1000,1632.73,0,not-in-family
1,Cust10001,Vehicle15893,Audi,A5,2006,N,2015-02-02,Multi-vehicle Collision,Side Collision,Total Loss,...,India,119122,114,2000-11-15,State1,100/300,1000,1255.19,0,not-in-family
2,Cust10002,Vehicle5152,Volkswagen,Jetta,1999,N,2015-01-15,Single Vehicle Collision,Side Collision,Minor Damage,...,India,119123,167,2001-02-12,State3,500/1000,617,1373.38,0,wife
3,Cust10003,Vehicle37363,Volkswagen,Jetta,2003,N,2015-01-19,Single Vehicle Collision,Side Collision,Minor Damage,...,India,119124,190,2005-04-11,State2,500/1000,722,1337.60,0,own-child
4,Cust10004,Vehicle28633,Toyota,CRV,2010,N,2015-01-09,Single Vehicle Collision,Rear Collision,Minor Damage,...,India,119125,115,1996-10-25,State2,100/300,500,1353.73,4279863,unmarried
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28831,Cust9993,Vehicle13568,Suburu,Impreza,2007,N,2015-01-24,Vehicle Theft,?,Trivial Damage,...,India,119114,321,1998-04-18,State1,500/1000,655,1276.01,0,unmarried
28832,Cust9994,Vehicle19810,Jeep,Wrangler,2003,N,2015-02-09,Single Vehicle Collision,Front Collision,Minor Damage,...,India,119115,237,2004-09-07,State2,100/300,1089,1273.38,0,unmarried
28833,Cust9996,Vehicle3339,Suburu,Legacy,2004,N,2015-01-28,Single Vehicle Collision,Rear Collision,Minor Damage,...,India,119117,392,1998-09-10,State3,250/500,787,1380.92,3448735,own-child
28834,Cust9997,Vehicle10240,Suburu,Forrestor,2004,N,2015-01-28,Single Vehicle Collision,Rear Collision,Minor Damage,...,India,119118,391,1998-08-10,State3,250/500,780,1389.29,3364301,not-in-family


In [416]:
df.columns

Index(['CustomerID', 'VehicleID', 'VehicleMake', 'VehicleModel', 'VehicleYOM',
       'ReportedFraud', 'DateOfIncident', 'TypeOfIncident', 'TypeOfCollission',
       'SeverityOfIncident', 'AuthoritiesContacted', 'IncidentState',
       'IncidentCity', 'IncidentAddress', 'IncidentTime', 'NumberOfVehicles',
       'PropertyDamage', 'BodilyInjuries', 'Witnesses', 'PoliceReport',
       'AmountOfTotalClaim', 'AmountOfInjuryClaim', 'AmountOfPropertyClaim',
       'AmountOfVehicleDamage', 'InsuredAge', 'InsuredZipCode',
       'InsuredGender', 'InsuredEducationLevel', 'InsuredOccupation',
       'InsuredHobbies', 'CapitalGains', 'CapitalLoss', 'Country'],
      dtype='object')

In [473]:
df['IncidentTime'].unique()

array([17, 10, 22,  7, 20, 18,  3,  5, 14, 16, 15, 13, 12,  9, 19,  4, 11,
        1,  8,  0,  6, 21, 23,  2, -5])

In [303]:
df_analysis = df.copy()

In [417]:
df_final = df_vehicle.merge(df_target, how='inner', on='CustomerID')

In [418]:
df_final = df_final.merge(df_claim, how='inner', on='CustomerID')

In [419]:
df_final = df_final.merge(df_demo, how='inner', on='CustomerID')

In [420]:
df_final = df_final.merge(df_policy, how='inner', on='CustomerID')

In [421]:
df_final

,CustomerID,VehicleMake,Years_from_man,ReportedFraud,SeverityOfIncident,IncidentState,IncidentCity,IncidentTime,NumberOfVehicles,PropertyDamage,...,Policy_CombinedSingleLimit,Policy_Deductible,PolicyAnnualPremium,UmbrellaLimit,YearFromPolicyCoverage,InsuredRelationship_not-in-family,InsuredRelationship_other-relative,InsuredRelationship_own-child,InsuredRelationship_unmarried,InsuredRelationship_wife
0,Cust10000,1.0,15,0,4,7,1,17,3,NaN,...,0.333333,1000,1632.73,0,25,1,0,0,0,0
1,Cust10001,1.0,17,0,4,7,5,10,3,1.0,...,0.333333,1000,1255.19,0,23,1,0,0,0,0
2,Cust10002,2.0,24,0,2,8,6,22,1,1.0,...,0.500000,617,1373.38,0,22,0,0,0,0,1
3,Cust10003,2.0,20,0,2,9,6,22,1,1.0,...,0.500000,722,1337.60,0,18,0,0,1,0,0
4,Cust10004,3.0,13,0,2,8,6,10,1,0.0,...,0.333333,500,1353.73,4279863,27,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28831,Cust9993,5.0,16,0,1,9,4,3,1,0.0,...,0.500000,655,1276.01,0,25,0,0,0,1,0
28832,Cust9994,13.0,20,0,2,8,3,17,1,1.0,...,0.333333,1089,1273.38,0,19,0,0,0,1,0
28833,Cust9996,5.0,19,0,2,9,7,1,1,1.0,...,0.500000,787,1380.92,3448735,25,0,0,1,0,0
28834,Cust9997,5.0,19,0,2,9,3,1,1,NaN,...,0.500000,780,1389.29,3364301,25,1,0,0,0,0


In [422]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28836 entries, 0 to 28835
Data columns (total 77 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   CustomerID                               28836 non-null  object 
 1   VehicleMake                              28786 non-null  float64
 2   Years_from_man                           28836 non-null  int64  
 3   ReportedFraud                            28836 non-null  int64  
 4   SeverityOfIncident                       28836 non-null  int64  
 5   IncidentState                            28836 non-null  int64  
 6   IncidentCity                             28836 non-null  int64  
 7   IncidentTime                             28836 non-null  int64  
 8   NumberOfVehicles                         28836 non-null  int64  
 9   PropertyDamage                           18377 non-null  float64
 10  BodilyInjuries                           28836

In [423]:
pd.set_option('display.max_rows', 500)
(df_final.isna().sum()/len(df_final))*100

CustomerID                                  0.000000
VehicleMake                                 0.173394
Years_from_man                              0.000000
ReportedFraud                               0.000000
SeverityOfIncident                          0.000000
IncidentState                               0.000000
IncidentCity                                0.000000
IncidentTime                                0.000000
NumberOfVehicles                            0.000000
PropertyDamage                             36.270634
BodilyInjuries                              0.000000
Witnesses                                   0.159523
PoliceReport                               34.002636
AmountOfTotalClaim                          0.173394
AmountOfInjuryClaim                         0.000000
AmountOfPropertyClaim                       0.000000
AmountOfVehicleDamage                       0.000000
MonthOfIncident                             0.000000
TypeOfIncident_Parked Car                   0.

In [424]:
df_final.drop(columns=['PropertyDamage','PoliceReport'],inplace=True)

In [425]:
df_final.dropna(axis=0, inplace=True)

In [426]:
(df_final.isna().sum()/len(df_final))*100

CustomerID                                 0.0
VehicleMake                                0.0
Years_from_man                             0.0
ReportedFraud                              0.0
SeverityOfIncident                         0.0
IncidentState                              0.0
IncidentCity                               0.0
IncidentTime                               0.0
NumberOfVehicles                           0.0
BodilyInjuries                             0.0
Witnesses                                  0.0
AmountOfTotalClaim                         0.0
AmountOfInjuryClaim                        0.0
AmountOfPropertyClaim                      0.0
AmountOfVehicleDamage                      0.0
MonthOfIncident                            0.0
TypeOfIncident_Parked Car                  0.0
TypeOfIncident_Single Vehicle Collision    0.0
TypeOfIncident_Vehicle Theft               0.0
TypeOfCollission_Rear Collision            0.0
TypeOfCollission_Side Collision            0.0
AuthoritiesCo

In [427]:
pd.reset_option('display.max_rows')

In [428]:
df_final.drop(columns='CustomerID', inplace=True)

# Model Building

In [429]:
def eval(y_train, y_train_pred, y_test, y_test_pred):
    print(f'Accuracy on training data:\t {accuracy_score(y_train, y_train_pred)}')
    print(f'Accuracy on test data:\t {accuracy_score(y_test, y_test_pred)} \n')
    
    print(f'Precision on training data:\t {precision_score(y_train, y_train_pred)}')
    print(f'Precision on test data:\t {precision_score(y_test, y_test_pred)} \n')
    
    print(f'Recall on training data:\t {recall_score(y_train, y_train_pred)}')
    print(f'Recall on test data:\t {recall_score(y_test, y_test_pred)} \n')
    
    print(f'F1 score on training data:\t {f1_score(y_train, y_train_pred)}')
    print(f'F1 score on test data:\t {f1_score(y_test, y_test_pred)} \n')

In [430]:
#set X and y
X = df_final.drop(['ReportedFraud'], axis=1)
y = df_final['ReportedFraud']

In [431]:
#split into train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [432]:
print(X_train.shape,'X Train')
print(X_test.shape,'X Test')
print(y_train.shape,'y Train')
print(y_test.shape,'y Test')

(22968, 73) X Train
(5742, 73) X Test
(22968,) y Train
(5742,) y Test


In [433]:
#checking target splits
print('Actual Data \n', df_final['ReportedFraud'].value_counts(normalize=True))
print('Train Data \n', y_train.value_counts(normalize=True))
print('Test Data \n', y_test.value_counts(normalize=True))

Actual Data 
 0    0.730024
1    0.269976
Name: ReportedFraud, dtype: float64
Train Data 
 0    0.729319
1    0.270681
Name: ReportedFraud, dtype: float64
Test Data 
 0    0.732846
1    0.267154
Name: ReportedFraud, dtype: float64


In [467]:
for i in X_train.columns:
    print(i)
    print(X_train[i].value_counts(),'\n')

VehicleMake
6.0     1939
7.0     1849
5.0     1837
10.0    1832
12.0    1719
8.0     1693
9.0     1688
14.0    1627
13.0    1583
1.0     1564
3.0     1564
2.0     1552
4.0     1331
11.0    1190
Name: VehicleMake, dtype: int64 

Years_from_man
16    1375
19    1355
13    1353
17    1336
18    1327
14    1305
15    1297
20    1285
12    1262
21    1255
23    1233
24    1219
22    1161
11    1061
25    1052
10    1014
26     893
9      725
27     669
28     443
8      348
Name: Years_from_man, dtype: int64 

SeverityOfIncident
2    8255
4    6513
3    6161
1    2039
Name: SeverityOfIncident, dtype: int64 

IncidentState
5    6321
7    5752
9    4917
8    2520
4    2360
6     649
3     449
Name: IncidentState, dtype: int64 

IncidentCity
2    3593
1    3471
4    3432
7    3320
3    3251
6    2952
5    2949
Name: IncidentCity, dtype: int64 

IncidentTime
16    1301
17    1273
15    1248
6     1239
14    1169
13    1128
4     1127
18    1126
5     1122
12    1088
7     1063
11    1038
19    

## Logistic Regression

In [435]:
lr = LogisticRegression()

In [436]:
lr.fit(X_train, y_train)

/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [437]:
y_train_pred = lr.predict(X_train)
y_test_pred = lr.predict(X_test)

In [438]:
print(classification_report(y_true=y_train, y_pred=y_train_pred))

              precision    recall  f1-score   support

           0       0.73      0.99      0.84     16751
           1       0.49      0.01      0.03      6217

    accuracy                           0.73     22968
   macro avg       0.61      0.50      0.44     22968
weighted avg       0.66      0.73      0.62     22968



In [439]:
print(classification_report(y_true=y_test, y_pred=y_test_pred))

              precision    recall  f1-score   support

           0       0.73      0.99      0.84      4208
           1       0.37      0.01      0.02      1534

    accuracy                           0.73      5742
   macro avg       0.55      0.50      0.43      5742
weighted avg       0.64      0.73      0.62      5742



In [440]:
eval(y_train, y_train_pred, y_test, y_test_pred)

Accuracy on training data:	 0.7291013584117032
Accuracy on test data:	 0.7311041448972484 

Precision on training data:	 0.4864864864864865
Precision on test data:	 0.3684210526315789 

Recall on training data:	 0.01447643557986167
Recall on test data:	 0.009126466753585397 

F1 score on training data:	 0.028116213683223996
F1 score on test data:	 0.017811704834605598 



# Random Forest

In [459]:
rf = RandomForestClassifier(random_state= 123, n_estimators=400, max_depth=15)

In [460]:
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=15, n_estimators=400, random_state=123)

In [461]:
y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)

In [462]:
print(classification_report(y_true=y_train, y_pred=y_train_pred))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97     16751
           1       0.99      0.83      0.90      6217

    accuracy                           0.95     22968
   macro avg       0.96      0.91      0.93     22968
weighted avg       0.95      0.95      0.95     22968



In [463]:
print(classification_report(y_true=y_test, y_pred=y_test_pred))

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4208
           1       0.93      0.71      0.80      1534

    accuracy                           0.91      5742
   macro avg       0.91      0.84      0.87      5742
weighted avg       0.91      0.91      0.90      5742



In [464]:
eval(y_train, y_train_pred, y_test, y_test_pred)

Accuracy on training data:	 0.9505834204110066
Accuracy on test data:	 0.9068268895855103 

Precision on training data:	 0.9860367253251722
Precision on test data:	 0.9258312020460358 

Recall on training data:	 0.8291780601576323
Recall on test data:	 0.7079530638852672 

F1 score on training data:	 0.9008300567933596
F1 score on test data:	 0.8023642408570372 

